ORDER:
- data_pulling, scrapy_VL, path_finding, data_processing, data_merging_VL

## TO USE: run all

final outputs:
## ABS
#### SA2 code to district names
- ../data/raw/ABS/SA2_TO_Name.csv
- code(int), name(String)

#### Digital boundary (AUS range, manually select VIC if needed)
- ../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp

#### LGA
- ../data/raw/LGA_to_SA2.csv
- SA2 (int), LGA (int), LGA_NAME22 (str)
- (SA2, LGA) pairs are unique

#### Estimated Resident Population (ERP) (2001 to 2021) (By SA2)
- ../data/raw/ABS/ERP/ERP.csv
- SA2 (int), year (int), population (int)
- SA2: interger code marking each district, e.g. 206041117 for Carlton

#### Median household income (weekly) (2021) (By SA2)
- ../data/raw/ABS/Household_income/Household_income.csv
- SA2 (int), year (int),  household_type (String), income_level (String), popultaion (int)

#### Population projection (2017 - 2066) (VIC overall)
- ../data/raw/ABS/Population/Population.csv
- ../data/curated/population_projection.csv
- year (int), popultaion (int)

#### Residential Property Price Index (1/2011 - 12/2021) (AUS)
- ../data/raw/ABS/Price_index/Price_index.csv
- year (int), month (int), price_index (int)

#### Immigration data, 2004 - 2019, Victoria only
- ../data/raw/ABS/immigration/immigration.csv
- year (int), immi_count (int)

#### Household debt income ratio measured on each two years In Millions (2009-2019) (Victoria only)
- ../data/raw/ABS/debt_income_ratio/debt_income_ratio.csv
- year (int), debt_ratio (float)


### ACARA
#### School location
- ../data/raw/ACARA/School_location/School_location.csv
- School Name (String), SA2 (int), Latitude (float), Longitude (float), School Type (String)

### RBA
#### Interest rate (2013 - 2021) (AUS)
- ../data/raw/rba/interest_rate/interest_rate.csv
- year(int), quarter(int), month(int), bond (float, risk free interest rate)
#### Exchange Rate (2010 March - 2022 June) AUS
- ../data/raw/rba/exchange_rate/exchange_rate.csv
- year (int), quarter (int), month (int), to_USD (float)


### Other Source
#### Median rent (1999 - 2021) BY sa2
- ../data/raw/DHHS/history_rent.csv
- SA2 (int), year (int), quarter (int), count (int), median (float)

In [1]:
from urllib.request import urlretrieve
import sys
import pandas as pd
import geopandas as gpd
import requests
import math
import zipfile
import os
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F



spark = (
    SparkSession.builder.appName("Assignment_2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
pd.options.display.float_format = "{:,.4f}".format

OUTPUT_DIR = "../data/raw/ABS/"

headers = {"accept": "text/csv"}

22/09/28 19:16:55 WARN Utils: Your hostname, Bruce-PC resolves to a loopback address: 127.0.1.1; using 172.22.204.2 instead (on interface eth0)
22/09/28 19:16:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/28 19:16:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/28 19:16:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
def pull_direct(url, output_dir, file_name):
    """
    use urlretrieve function to directly pull data from given url and save it 
        to path: {output_dir}{file_name}
    url: the String url which needs to be pulled from
    output_dir: the String output folder directory, automatically create if not exist
    file_name: the String file name of the file needed to be pulled from
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        urlretrieve(url, f"{output_dir}{file_name}")
        print(
            f"Request succeed: pulling from{url}\nFile saved in: {output_dir}{file_name}"
        )
    except Exception as e:
        print(f"********\nRequest failure: ")
        print(e)
        print("********")


def write_file(output_dir, file_name, content, mod="w"):
    """
    write given content to local file at: {output_dir}{file_name} with mode: {mod}
    output_dir: the String output folder directory, automatically create if not exist
    file_name: the String file name used to save file
    content: expecting objects which can be written to file with open function
    mod: String of writing mode code used in writing file
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(f"{output_dir}{file_name}", mod) as f:
            f.write(content)
    except Exception as e:
        print(f"****** Writing file failure: {output_dir}{file_name}")
        print(e)
        print("******")
        return False
    return True



## ABS

#### SA2 code to district names
- ../data/raw/ABS/SA2_TO_Name.csv
- code(int), name(String)

In [5]:
### Pull SA2 match table
url = f"https://www.abs.gov.au/statistics/standards/australian-statistical\
-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads\
/allocation-files/SA2_2021_AUST.xlsx"
file_name = "SA2_TO_Name.xlsx"
pull_direct(url, OUTPUT_DIR, file_name)

# Select victoria data then save
vic_df = pd.read_excel(f"{OUTPUT_DIR}{file_name}")
vic_df = vic_df.loc[vic_df["STATE_NAME_2021"] == "Victoria"]
vic_df = vic_df[["SA2_CODE_2021", "SA2_NAME_2021"]].rename(
    columns={"SA2_CODE_2021": "code", "SA2_NAME_2021": "name"}
)
vic_df.to_csv(f"{OUTPUT_DIR}SA2_TO_Name.csv")

Request succeed: pulling fromhttps://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files/SA2_2021_AUST.xlsx
File saved in: ../data/raw/ABS/SA2_TO_Name.xlsx


#### Digital boundary (AUS range, manually select VIC if needed)
- ../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp

In [6]:
### pull shape file
url = f"https://www.abs.gov.au/statistics/standards/australian-statistical\
-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads\
/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"
file_name = "SA2_2021_AUST_SHP_GDA2020.zip"
pull_direct(url, OUTPUT_DIR, file_name)

# unzip zip file
with zipfile.ZipFile(f"{OUTPUT_DIR}{file_name}", "r") as zip_ref:
    zip_ref.extractall(f"{OUTPUT_DIR}digitalBoundary/")


Request succeed: pulling fromhttps://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip
File saved in: ../data/raw/ABS/SA2_2021_AUST_SHP_GDA2020.zip


### LGA
- ../data/raw/LGA_to_SA2.csv
- SA2 (int), LGA (int), LGA_NAME22 (str)
- (SA2, LGA) pairs are unique

In [7]:
output_dir = "../data/raw/ABS/"

### pull shape file
url = f"https://www.abs.gov.au/statistics/standards/australian-statistical\
-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/\
digital-boundary-files/LGA_2022_AUST_GDA2020_SHP.zip"
file_name = "LGA_2022_AUST_GDA2020_SHP.zip"
pull_direct(url, output_dir, file_name)

# unzip zip file
with zipfile.ZipFile(f"{output_dir}{file_name}", "r") as zip_ref:
    zip_ref.extractall(f"{output_dir}LGA/")


Request succeed: pulling fromhttps://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/LGA_2022_AUST_GDA2020_SHP.zip
File saved in: ../data/raw/ABS/LGA_2022_AUST_GDA2020_SHP.zip


In [8]:
output_dir = "../data/raw/ABS/"
## read data
lga_gdf = gpd.read_file(f"{output_dir}LGA/LGA_2022_AUST_GDA2020.shp")
sa2_gdf = gpd.read_file(f"../data/raw/ABS/digitalBoundary/SA2_2021_AUST_GDA2020.shp")

## feature selection for sa2 gdf
sa2_gdf = sa2_gdf.loc[sa2_gdf["STE_NAME21"] == "Victoria"].dropna()
sa2_gdf = sa2_gdf[["SA2_CODE21", "geometry"]]
sa2_gdf["SA2_CODE21"] = sa2_gdf["SA2_CODE21"].astype("int64")
sa2_gdf = sa2_gdf.rename(columns={"SA2_CODE21": "SA2"}).set_index("SA2")


## feature selection
lga_gdf = lga_gdf.loc[lga_gdf["STE_NAME21"] == "Victoria"][["LGA_CODE22", "LGA_NAME22", "geometry"]]
lga_gdf = lga_gdf.rename(columns={"LGA_CODE22": "LGA"}).astype({"LGA": int})
lga_gdf = lga_gdf.set_index("LGA")

## save file
lga_gdf.to_file(f"{output_dir}LGA/LGA.shp")

print(lga_gdf.shape)
print(lga_gdf.columns)
print(lga_gdf.head())

(82, 2)
Index(['LGA_NAME22', 'geometry'], dtype='object')
       LGA_NAME22                                           geometry
LGA                                                                 
20110      Alpine  POLYGON ((146.67058 -36.56827, 146.67056 -36.5...
20260      Ararat  POLYGON ((142.71709 -37.15729, 142.71705 -37.1...
20570    Ballarat  POLYGON ((143.86292 -37.38007, 143.85880 -37.4...
20660     Banyule  POLYGON ((145.08875 -37.69135, 145.08926 -37.6...
20740  Bass Coast  MULTIPOLYGON (((145.10992 -38.51956, 145.10976...


In [9]:
join_gdf = gpd.sjoin(sa2_gdf, lga_gdf, how="left")
join_gdf = join_gdf.rename(columns={"index_right": "LGA"})
join_gdf = join_gdf.astype({"LGA": int})

join_df = pd.DataFrame(join_gdf.drop(columns = "geometry"))
join_df.to_csv("../data/raw/LGA_to_SA2.csv")
print(join_df.head())

             LGA     LGA_NAME22
SA2                            
201011001  22490  Golden Plains
201011001  20570       Ballarat
201011002  20570       Ballarat
201011005  22490  Golden Plains
201011005  25150      Moorabool


## ABS Data

#### Estimated Resident Population (ERP) (2001 to 2021) (By SA2)
- ../data/raw/ABS/ERP/ERP.csv
- SA2 (int), year (int), population (int)
- SA2: interger code marking each district, e.g. 206041117 for Carlton

In [10]:
### Pull ERP data
url = "https://api.data.abs.gov.au/data/ABS,ABS_ANNUAL_ERP_ASGS2021,1.2.0/.\
SA2..A?startPeriod=2010&endPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url, headers=headers)
write_file(f"{OUTPUT_DIR}ERP/", "ERP_raw.csv", response.text)

True

In [11]:
# read file
erp_sdf = spark.read.csv(f"{OUTPUT_DIR}ERP/ERP_raw.csv", header=True)
print("Before selection:")
erp_sdf.show(2, vertical=True, truncate=100)

# feature selection / rename / reset datatype
erp_sdf = erp_sdf[["ASGS_2021", "TIME_PERIOD", "OBS_VALUE"]]
erp_sdf = (
    erp_sdf.withColumnRenamed("ASGS_2021", "SA2")
    .withColumn("SA2", F.col("SA2").cast("int"))
    .withColumnRenamed("TIME_PERIOD", "year")
    .withColumn("year", F.col("year").cast("int"))
    .withColumnRenamed("OBS_VALUE", "population")
    .withColumn("population", F.col("population").cast("int"))
)

# Filter: victoria SA2
vic_df = pd.read_csv(f"{OUTPUT_DIR}SA2_TO_Name.csv")
vic_sa2 = vic_df["code"].tolist()
erp_sdf = erp_sdf.filter(F.col("SA2").isin(vic_sa2))
erp_sdf = erp_sdf.sort(F.col("year").desc())

# save file
print("After selection:")
erp_sdf.show(1, vertical=True, truncate=100)
erp_sdf.write.option("header", True).mode("overwrite").csv(f"{OUTPUT_DIR}ERP/ERP.csv")


Before selection:
-RECORD 0------------------------------------------
 DATAFLOW     | ABS:ABS_ANNUAL_ERP_ASGS2021(1.2.0) 
 MEASURE      | ERP                                
 REGION_TYPE  | SA2                                
 ASGS_2021    | 101021010                          
 FREQ         | A                                  
 TIME_PERIOD  | 2010                               
 OBS_VALUE    | 4813                               
 UNIT_MEASURE | PSNS                               
 OBS_STATUS   | null                               
 OBS_COMMENT  | null                               
-RECORD 1------------------------------------------
 DATAFLOW     | ABS:ABS_ANNUAL_ERP_ASGS2021(1.2.0) 
 MEASURE      | ERP                                
 REGION_TYPE  | SA2                                
 ASGS_2021    | 101021010                          
 FREQ         | A                                  
 TIME_PERIOD  | 2011                               
 OBS_VALUE    | 4951                          

#### Median household income (weekly) (2021) (By SA2)
- ../data/raw/ABS/Household_income/Household_income.csv
- SA2 (int), year (int),  household_type (String), income_level (String), popultaion (int)

In [12]:
### pull median household income data
url = f"https://api.data.abs.gov.au/data/ABS,C21_G02_SA2,1.0.0/4..SA2.2?startPeriod=2021&dimensionAtObservation=AllDimensions"
response = requests.get(url, headers=headers)
write_file(f"{OUTPUT_DIR}Household_income/", "Household_income_raw.csv",
            response.text)

True

In [13]:
# read file
income_sdf = spark.read.csv(
    f"{OUTPUT_DIR}Household_income/Household_income_raw.csv", header=True
)
print("Before selection:")
income_sdf.show(2, vertical=True, truncate=100)

# filt data / rename
income_sdf = income_sdf[["REGION", "OBS_VALUE"]]
income_sdf = (
    income_sdf.withColumnRenamed("REGION", "SA2")
    .withColumn("SA2", F.col("SA2").cast("int"))
    .withColumnRenamed("OBS_VALUE", "median_income")
    .withColumn("median_income", F.col("median_income").cast("int"))
)

# save data
print("Before after:")
income_sdf.show(2, vertical=True, truncate=100)
income_sdf.write.option("header", True).mode("overwrite").csv(
    f"{OUTPUT_DIR}Household_income/Household_income.csv"
)

Before selection:
-RECORD 0-----------------------------
 DATAFLOW    | ABS:C21_G02_SA2(1.0.0) 
 MEDAVG      | 4                      
 REGION      | 213051589              
 REGION_TYPE | SA2                    
 STATE       | 2                      
 TIME_PERIOD | 2021                   
 OBS_VALUE   | 1862                   
-RECORD 1-----------------------------
 DATAFLOW    | ABS:C21_G02_SA2(1.0.0) 
 MEDAVG      | 4                      
 REGION      | 209041437              
 REGION_TYPE | SA2                    
 STATE       | 2                      
 TIME_PERIOD | 2021                   
 OBS_VALUE   | 1979                   
only showing top 2 rows

Before after:
-RECORD 0------------------
 SA2           | 213051589 
 median_income | 1862      
-RECORD 1------------------
 SA2           | 209041437 
 median_income | 1979      
only showing top 2 rows



Population projection (2017 - 2066) (VIC overall)
- ../data/raw/ABS/Population/Population.csv
- year (int), popultaion (int)

In [14]:
### pull popultation projection data
url = "https://api.data.abs.gov.au/data/ABS,POP_PROJ_REGION_2012_2061,\
1.0.0/2.3.TT.1.1.1.1.A?startPeriod=2017&dimensionAtObservation=AllDimensions"
response = requests.get(url, headers=headers)
write_file(f"{OUTPUT_DIR}Population/", "Population_raw.csv", response.text)


True

In [15]:
# read data
population_sdf = spark.read.csv(
    f"{OUTPUT_DIR}Population/Population_raw.csv", header=True
)
print("Before selection:")
population_sdf.show(2, vertical=True, truncate=100)

# feature selection / rename / reset datatype
population_sdf = population_sdf[["TIME_PERIOD", "OBS_VALUE"]]
population_sdf = (
    population_sdf.withColumnRenamed("TIME_PERIOD", "year")
    .withColumn("year", F.col("year").cast("int"))
    .withColumnRenamed("OBS_VALUE", "popultaion")
    .withColumn("popultaion", F.col("popultaion").cast("int"))
)

# save modified file
print("After selection:")
population_sdf.show(2, vertical=True, truncate=100)
population_sdf.write.option("header", True).mode("overwrite").csv(
    f"{OUTPUT_DIR}Population/Population.csv"
)
population_sdf.write.option("header", True).mode("overwrite").csv(
    f"../data/curated/population_projection.csv"
)

Before selection:
-RECORD 0--------------------------------------------
 DATAFLOW     | ABS:POP_PROJ_REGION_2012_2061(1.0.0) 
 REGION       | 2                                    
 SEX_ABS      | 3                                    
 AGE          | TT                                   
 FERTILITY    | 1                                    
 MORTALITY    | 1                                    
 NOM          | 1                                    
 NIM          | 1                                    
 FREQUENCY    | A                                    
 TIME_PERIOD  | 2017                                 
 OBS_VALUE    | 6321648                              
 UNIT_MEASURE | PSNS                                 
 OBS_STATUS   | null                                 
 OBS_COMMENT  | null                                 
-RECORD 1--------------------------------------------
 DATAFLOW     | ABS:POP_PROJ_REGION_2012_2061(1.0.0) 
 REGION       | 2                                    
 SEX_ABS  

#### Residential Property Price Index (1/2011 - 12/2021) (AUS)
- ../data/raw/ABS/Price_index/Price_index.csv
- year (int), month (int), price_index (int)

In [16]:
output_dir = f"../data/raw/ABS/"

### Pull Price_index data
url = "https://api.data.abs.gov.au/data/ABS,RPPI,1.0.0/1.3.2GMEL.Q?startPeriod=2011-Q1&endPeriod=2021-Q4&dimensionAtObservation=AllDimensions"
response = requests.get(url, headers=headers)
write_file(f"{output_dir}Price_index/", "Price_index_raw.csv", response.text)


True

In [14]:
output_dir = f"../data/raw/ABS/"
# read file
price_sdf = spark.read.csv(f"{output_dir}Price_index/Price_index_raw.csv", header=True)
print("Before selection:")
price_sdf.show(1, vertical=True, truncate=100)

# feature selection / rename / reset datatype
price_sdf = price_sdf[["TIME_PERIOD", "OBS_VALUE"]]
price_sdf = (
    price_sdf.withColumnRenamed("TIME_PERIOD", "year")
    .withColumnRenamed("OBS_VALUE", "price_index")
    .withColumn("price_index", F.col("price_index").cast("int"))
)
price_sdf = price_sdf.withColumn("quarter", F.substring("year", 7, 7).cast("int"))
price_sdf = price_sdf.withColumn("year", F.substring("year", 1, 4).cast("int"))
price_sdf = price_sdf.dropna().dropDuplicates().sort("year", "quarter")

# save file
print("After selection:")
price_sdf.show(2, vertical=True, truncate=100)
price_sdf.write.option("header", True).mode("overwrite").csv(
    f"{output_dir}Price_index/Price_index.csv"
)


Before selection:
-RECORD 0------------------------
 DATAFLOW      | ABS:RPPI(1.0.0) 
 MEASURE       | 1               
 PROPERTY_TYPE | 3               
 REGION        | 2GMEL           
 FREQ          | Q               
 TIME_PERIOD   | 2011-Q1         
 OBS_VALUE     | 104.7           
 UNIT_MEASURE  | IN              
 OBS_STATUS    | null            
 OBS_COMMENT   | null            
only showing top 1 row

After selection:
-RECORD 0-----------
 year        | 2011 
 price_index | 104  
 quarter     | 1    
-RECORD 1-----------
 year        | 2011 
 price_index | 103  
 quarter     | 2    
only showing top 2 rows



### Immigration data, 2004 - 2019, Victoria only
- ../data/raw/ABS/immigration/immigration.csv
- year (int), immi_count (int)

In [18]:
output_dir = f"../data/raw/ABS/immigration/"
### Pull SA2 match table
url = f"https://www.abs.gov.au/statistics/people/population/overseas-migration/2020-21/34070DO002_202021.xlsx"
file_name = "immigration.xlsx"
pull_direct(url, output_dir, file_name)

Request succeed: pulling fromhttps://www.abs.gov.au/statistics/people/population/overseas-migration/2020-21/34070DO002_202021.xlsx
File saved in: ../data/raw/ABS/immigration/immigration.xlsx


In [19]:
# Select victoria data then save
immi_df = pd.read_excel(f"../data/raw/ABS/immigration/immigration.xlsx", sheet_name="Table 2.3")


# reformate data
immi_df = immi_df.loc[range(13, immi_df.shape[0])].loc[[13, 29]]
immi_df = immi_df.reset_index().set_axis(["year", "immi_count"])
immi_df = immi_df.drop(columns=[
    "            Australian Bureau of Statistics", 
    'Unnamed: 1', 
    'Unnamed: 2',
    "index"])
immi_df = immi_df.T
immi_df["year"] = immi_df["year"].apply(lambda x: int(x[:4]))
immi_df["immi_count"] = immi_df["immi_count"].apply(lambda x: int(x))
immi_df = immi_df.reset_index().drop(columns=["index"])
immi_df = immi_df.loc[immi_df.index.tolist()[:-1]]

immi_df.to_csv(f"../data/raw/ABS/immigration/immigration.csv", index_label="index")
print(immi_df.columns)
print(immi_df.tail())

Index(['year', 'immi_count'], dtype='object')
    year  immi_count
11  2015      140220
12  2016      159630
13  2017      161170
14  2018      168310
15  2019      150630


### Household debt income ratio measured on each two years In Millions (2009-2019) (Victoria only)
- ../data/raw/ABS/debt_income_ratio/debt_income_ratio.csv
- year (int), debt_ratio (float)

In [20]:

output_dir = f"../data/raw/ABS/debt_income_ratio/"
### Pull SA2 match table
url = f"https://www.abs.gov.au/statistics/economy/finance/household-income-and-wealth-australia/2019-20/3.%20Income%2C%20wealth%20and%20debt.xlsx"
file_name = "debt_income_ratio.xlsx"
pull_direct(url, output_dir, file_name)

Request succeed: pulling fromhttps://www.abs.gov.au/statistics/economy/finance/household-income-and-wealth-australia/2019-20/3.%20Income%2C%20wealth%20and%20debt.xlsx
File saved in: ../data/raw/ABS/debt_income_ratio/debt_income_ratio.xlsx


In [21]:
# Select victoria data then save
ratio_df = pd.read_excel(f"../data/raw/ABS/debt_income_ratio/debt_income_ratio.xlsx", sheet_name="Table 3.5")

# select needed data
ratio_df = ratio_df.loc[[4, 67]].set_axis(["time", "ratio"])
ratio_df = ratio_df.T.reset_index().drop(["index"], axis=1)
ratio_df = ratio_df.loc[range(8, 14)]

# reformate data
time_list = [int(i[:4]) for i in ratio_df["time"].to_list()]
time_list = sorted([i+1 for i in time_list] + time_list)[:-1]
rate_list = ratio_df["ratio"].to_list()
rate_list = [[rate_list[i], (rate_list[i] + rate_list[i+1])/2] for i in range(len(rate_list)-1)] + [[rate_list[-1]]]
rate_list = [i for l in rate_list for i in l]
ratio_df = pd.DataFrame(list(zip(time_list, rate_list)), columns=["year", "debt_ratio"])

# save to file
ratio_df.to_csv(f"../data/raw/ABS/debt_income_ratio/debt_income_ratio.csv", index_label="index")
print(ratio_df.columns)
print(ratio_df.head(20))

Index(['year', 'debt_ratio'], dtype='object')
    year  debt_ratio
0   2009      0.9700
1   2010      0.9100
2   2011      0.8500
3   2012      0.8550
4   2013      0.8600
5   2014      0.8550
6   2015      0.8500
7   2016      1.0150
8   2017      1.1800
9   2018      1.1450
10  2019      1.1100


## ACARA

#### School location
- ../data/raw/ACARA/School_location/School_location.csv
- School Name (String), SA2 (int), Latitude (float), Longitude (float), School Type (String)

In [22]:
output_dir = f"../data/raw/ACARA/"

# pull school location file
url = "https://www.acara.edu.au/docs/default-source/default-document-\
library/school-location-2021e23a2f404c94637ead88ff00003e0139.xlsx\
?sfvrsn=51ae4c07_0"
response = requests.get(url)
write_file(f"{output_dir}", "School_location_raw.xlsx",
            response.content, "wb")
# pull_direct(url, output_dir, f"School_location.xlsx")

True

In [23]:
output_dir = f"../data/raw/ACARA/"

# prepare boundary gdf
boundary_df = pd.DataFrame(gpd.read_file(f"../data/raw/ABS/digitalBoundary/\
SA2_2021_AUST_GDA2020.shp")).rename(columns={"SA2_CODE21": "SA2"})
boundary_df = boundary_df.loc[boundary_df["STE_NAME21"] == "Victoria"][["SA2"]]
boundary_df["SA2"] = pd.to_numeric(boundary_df["SA2"])

# feature selection / VIC filtering
school_df = pd.read_excel(
    f"{output_dir}School_location_raw.xlsx",
    sheet_name="SchoolLocations 2021",
)[["School Name", "Statistical Area 2", "State", "Latitude", 
    "Longitude", "School Type"]]
school_df = school_df.loc[school_df["School Type"] != "Special"]
school_df = school_df.loc[school_df["State"] == "VIC"]
school_df = school_df.rename(columns={"Statistical Area 2": "SA2"})
school_df = school_df.drop(columns="State")
school_df = school_df.dropna()

# right join
school_df = pd.merge(school_df, boundary_df, on="SA2", how="right")

# add index
school_df["school_id"] = range(school_df.shape[0])
school_df = school_df.reset_index().set_index("school_id").drop(["index"], axis=1)

print(school_df.head())
print(school_df.shape)

# save filtered data
school_df = school_df.dropna().drop_duplicates()
school_df.to_csv(f"{output_dir}School_location.csv")


                                     School Name        SA2  Latitude  \
school_id                                                               
0                       Alfredton Primary School  201011001  -37.5550   
1                          St Thomas More School  201011001  -37.5535   
2                  Siena Catholic Primary School  201011001  -37.5462   
3                           Lucas Primary School  201011001  -37.5481   
4          Ballarat Primary School (Dana Street)  201011002  -37.5640   

           Longitude School Type  
school_id                         
0           143.8017     Primary  
1           143.7999     Primary  
2           143.7801     Primary  
3           143.7804     Primary  
4           143.8539     Primary  
(2402, 5)


# RBA

### Interest rate (2013 - 2021) (AUS)
- ../data/raw/rba/interest_rate/interest_rate.csv
- year(int), quarter(int), month(int), bond (float, risk free interest rate)

In [24]:
output_dir = f"../data/raw/rba/"

### Pull Price_index data
url = "https://www.rba.gov.au/statistics/tables/csv/f2.1-data.csv?v=2022-09-08-09-00-18"
response = requests.get(url, headers=headers)
write_file(f"{output_dir}interest_rate/", "interest_rate_raw.csv", response.text)


True

In [25]:
output_dir = f"../data/raw/rba/"
# read file
interest_df = pd.read_csv(f"{output_dir}interest_rate/interest_rate_raw.csv")

# data cleaning and renaming
col_names = ["time"] + list(interest_df.iloc[[0]].values[0][1:])
interest_df = interest_df.iloc[11:]
interest_df.columns = col_names
interest_df = interest_df[["time", "Commonwealth Government 3 year bond"]]

# data selection (fourth quarter bond)
interest_df.columns = ["time", "bond"]
interest_df["time"] = pd.to_datetime(interest_df["time"])
interest_df["year"] = interest_df["time"].dt.year
interest_df["month"] = interest_df["time"].dt.month
interest_df["quarter"] = interest_df["time"].dt.quarter
interest_df["bond"] = interest_df["bond"].astype("float")
interest_df = interest_df.reset_index().drop("index", axis=1)
interest_df = interest_df[["year", "quarter", "month", "bond"]]

print(interest_df.head())
interest_df.to_csv(f"{output_dir}interest_rate/interest_rate.csv", index_label="index")


   year  quarter  month   bond
0  2013        3      7 2.6900
1  2013        3      8 2.6900
2  2013        3      9 2.9000
3  2013        4     10 2.9500
4  2013        4     11 3.0300


### Exchange Rate (2010 March - 2022 June) AUS
- ../data/raw/rba/exchange_rate/exchange_rate.csv
- year (int), quarter (int), month (int), to_USD (float)

In [26]:
output_dir = f"../data/raw/rba/"

### Pull Price_index data
url = "https://www.rba.gov.au/statistics/tables/csv/f11-data.csv"
response = requests.get(url, headers=headers)
write_file(f"{output_dir}exchange_rate/", "exchange_rate_raw.csv", response.text)


True

In [27]:
output_dir = f"../data/raw/rba/"
# read file
exchange_df = pd.read_csv(f"{output_dir}exchange_rate/exchange_rate_raw.csv")

# data reformate
exchange_df = exchange_df.loc[range(10, exchange_df.shape[0])]
exchange_df = exchange_df.rename({"F11 EXCHANGE RATES ": "year", "Unnamed: 1": "to_USD"}, axis=1)
exchange_df = exchange_df[["year", "to_USD"]]

# # feature engineering
exchange_df["year"] = exchange_df["year"].apply(lambda x: pd.to_datetime(x))
exchange_df = exchange_df.astype({"to_USD": float})
exchange_df['quarter'] = exchange_df['year'].dt.quarter
exchange_df['month'] = exchange_df['year'].dt.month
exchange_df['year'] = exchange_df['year'].dt.year
exchange_df = exchange_df.loc[(exchange_df["month"].isin([3, 6, 9, 12]))]
exchange_df = exchange_df.reset_index().drop(["index"], axis=1)
exchange_df = exchange_df[['year', "quarter", 'month', 'to_USD']]

# save data
print(exchange_df.shape)
print(exchange_df.columns)
print(exchange_df.tail(15))
exchange_df.to_csv(f"{output_dir}exchange_rate/exchange_rate.csv", index_label="index")


(50, 4)
Index(['year', 'quarter', 'month', 'to_USD'], dtype='object')
    year  quarter  month  to_USD
35  2018        4     12  0.7058
36  2019        1      3  0.7087
37  2019        2      6  0.7013
38  2019        3      9  0.6749
39  2019        4     12  0.7006
40  2020        1      3  0.6175
41  2020        2      6  0.6863
42  2020        3      9  0.7108
43  2020        4     12  0.7702
44  2021        1      3  0.7602
45  2021        2      6  0.7518
46  2021        3      9  0.7206
47  2021        4     12  0.7256
48  2022        1      3  0.7482
49  2022        2      6  0.6889


## Median rent (1999 - 2021) BY sa2
- ../data/raw/DHHS/history_rent.csv
- SA2 (int), year (int), quarter (int), count (int), median (float)

In [28]:
output_dir = f"../data/raw/DHHS/"
file_name = "history_rent_quarterly.xlsx"
url = f"https://www.dhhs.vic.gov.au/quarterly-median\
-rents-local-government-area-march-quarter-2021-excel"

pull_direct(url, output_dir, file_name)

Request succeed: pulling fromhttps://www.dhhs.vic.gov.au/quarterly-median-rents-local-government-area-march-quarter-2021-excel
File saved in: ../data/raw/DHHS/history_rent_quarterly.xlsx


In [29]:
# read file
hisrent_df = pd.read_excel(f"../data/raw/DHHS/history_rent_quarterly.xlsx", sheet_name="All Properties")

hisrent_df = hisrent_df.loc[range(3, 92)]
hisrent_df = hisrent_df.loc[hisrent_df["Unnamed: 1"] != "Group Total"].T
hisrent_df = hisrent_df.reset_index().drop(["index"], axis=1)
hisrent_df = hisrent_df.T.reset_index().drop([0, "index"], axis=1).dropna()


print(hisrent_df.shape)
print(hisrent_df.head())

(80, 177)
               1     2    3     4    5     6    7     8    9     10   ...  \
0      Colac-Otway   100  114    78  115    81  115    97  115    91  ...   
1      Corangamite    35   90    45   90    47  100    50  100    40  ...   
2          Glenelg   124  110   119  100   120  100   141  105   129  ...   
3  Greater Geelong  1371  135  1326  135  1406  140  1510  140  1316  ...   
4            Moyne    43  130    38  120    45  120    58  120    42  ...   

    168  169   170  171   172  173   174  175   176  177  
0   131  320    87  320   130  335   112  328    86  335  
1    63  300    57  270    76  280    52  290    56  315  
2    93  300    98  290    88  300    75  320    83  330  
3  2208  380  1811  370  2063  375  1970  380  1809  400  
4    53  360    50  300    45  320    41  370    45  350  

[5 rows x 177 columns]


In [30]:
# reformate dataframe
rent_df = pd.DataFrame(columns=["LGA", "year", "quarter", "count", "median"])
num_col = hisrent_df.shape[1]
year_list = range(1999, 2022)
quarter_list = [1, 2, 3, 4]
sec_list = [[i, j] for i in year_list for j in quarter_list][1:-3]


k=0
# add record line by line
for i in range(hisrent_df.shape[0]):
    record = hisrent_df.loc[i].to_list()
    j = 1
    while j < num_col:
        rent_df.loc[k] = [record[0], sec_list[int((j-1)/2)][0], sec_list[int((j-1)/2)][1], record[j], record[j+1]]
        j+=2
        k+=1

print(rent_df.head())
rent_df.to_csv("../data/raw/DHHS/history_rent_LGA.csv", index_label="index")

           LGA  year  quarter count median
0  Colac-Otway  1999        2   100    114
1  Colac-Otway  1999        3    78    115
2  Colac-Otway  1999        4    81    115
3  Colac-Otway  2000        1    97    115
4  Colac-Otway  2000        2    91    120


In [31]:
hisrent_df = pd.read_csv(f"../data/raw/DHHS/history_rent_LGA.csv")
conv_df = pd.read_csv(f"../data/raw/LGA_to_SA2.csv")

rent_df = pd.merge(hisrent_df, conv_df, left_on="LGA", right_on = "LGA_NAME22", how="right")
rent_df = rent_df.replace("-", pd.NA)
rent_df = rent_df[['SA2', 'year', 'quarter', 'count', 'median']].dropna()
rent_df = rent_df.astype({
    'SA2': int,
    'year': int, 
    'quarter': int, 
    'count': int,
    'median': float
})
rent_df = rent_df.groupby(["SA2", "year", "quarter"]).agg({"count": "sum", "median": "mean"}).reset_index()
rent_df = rent_df.astype({
    'count': int,
    'median': float
})
rent_df.to_csv("../data/raw/DHHS/history_rent.csv", header=True, index_label="index")

print(rent_df.head())
print(rent_df.columns)

         SA2  year  quarter  count   median
0  201011001  1999        2    687 136.5000
1  201011001  1999        3    682 132.5000
2  201011001  1999        4    716 130.5000
3  201011001  2000        1   1114 139.0000
4  201011001  2000        2    631 132.5000
Index(['SA2', 'year', 'quarter', 'count', 'median'], dtype='object')


By Junhua Liu for study use only